Combined with the API reference docs, this notebook gives an overview of `voeventdb.remote`, for those who are already familiar with Python, VOEvents, and the basic ideas around web-based API usage. A gentler introduction can be found in the rest of the tutorial notebooks.

In [ ]:
from __future__ import print_function
import voeventdb.remote as vr
import voeventdb.remote.apiv1 as apiv1
from voeventdb.remote.apiv1 import FilterKeys, OrderValues
from voeventdb.remote.helpers import Synopsis
from datetime import datetime
import pytz

In `voeventdb.remote` endpoints are represented by functions, and can be found under the `apiv1` namespace:

In [ ]:
#Get a count of VOEvents in the database
apiv1.count()

In [ ]:
#List the IVORNs of the 2 VOEvents most recently inserted into the database
apiv1.list_ivorn(order=OrderValues.id_desc,
                 n_max=2)

We can apply filters to all endpoints except those returning a single packet. If no filters are passed then the result-set is effectively the entire database. 

Filter-sets are defined as dictionaries:

In [ ]:
#Filter to only 'observation' packets authored since 2015/12/01.
my_filters = {
    FilterKeys.role:'observation', 
    FilterKeys.authored_since:datetime(2015,12,1,tzinfo=pytz.UTC),
}

In [ ]:
#The `map_` endpoints return dictionaries summarising the matching result-set:
apiv1.map_stream_count(filters=my_filters)

In [ ]:
#Get the IVORN of the most recently authored SWIFT GRB alert:
my_filters = {
    FilterKeys.role:'observation', 
    FilterKeys.ivorn_contains:'BAT_GRB', #case sensitive
}

swift_bat_grb_list = apiv1.list_ivorn(
    filters=my_filters,
    order=OrderValues.author_datetime_desc,
    n_max=1
)
swift_bat_grb_ivorn = swift_bat_grb_list[0]
swift_bat_grb_ivorn

To get more detail on a VOEvent packet, you can use the 'packet_synopsis' function:

In [ ]:
# Retrieve a 'synopsis' (nested dictionary) for the VOEvent packet,
grb_nested_dict = apiv1.packet_synopsis(swift_bat_grb_ivorn)
# And convert it to a user-friendly class-object:
grb_synopsis = Synopsis(grb_nested_dict)
print(grb_synopsis.ivorn)
print(grb_synopsis.author_ivorn)
print(grb_synopsis.author_datetime)
print(grb_synopsis.sky_events)

voeventdb.remote makes use of Astropy [coordinates](http://astropy.readthedocs.org/en/stable/coordinates/index.html) . So for example, the [Synopsis](http://voeventdbremote.readthedocs.org/en/stable/reference/index.html#voeventdb.remote.helpers.Synopsis) helper class parses the event-position co-ordinates into an Astropy [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html):

In [ ]:
sky_event=grb_synopsis.sky_events[0]

sky_event.position

In [ ]:
print(sky_event.position.ra.deg)
print(sky_event.position.ra.hms)

We can search for VOEvents with nearby co-ordinates to the Swift event we just looked up:

In [ ]:
from astropy.coordinates import SkyCoord, Angle
my_filters = {
    FilterKeys.role:'observation', 
    FilterKeys.cone: (sky_event.position, Angle(1.0, unit='deg'))
    }
apiv1.map_stream_count(my_filters)

Or we run a search at co-ordinates of our choice (see the SkyCoord [docs](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) for parseable formats):

In [ ]:
from astropy.coordinates import SkyCoord, Angle

chosen_position = SkyCoord(45,45, unit='deg')
radius = Angle(1.0, unit='deg')
my_filters = {
    FilterKeys.role:'observation', 
    FilterKeys.cone: (chosen_position,radius)
    }
apiv1.map_stream_count(my_filters)

Note we can also retrieve the original XML for the packet (and save it to disk, etc):

In [ ]:
raw_xml = apiv1.packet_xml(swift_bat_grb_ivorn)
print(raw_xml[:72])
print(' ... ')
with open('swift_voevent.xml','w') as f:
    f.write(raw_xml)